In [1]:
import warnings
warnings.filterwarnings('ignore')

import scipy.io as sc
import numpy as np
import random
import sys

from sklearn import preprocessing

from keras import backend as K

from keras.models import load_model

from xgboost import XGBClassifier

import pickle

Using TensorFlow backend.


In [2]:
# this function is used to transfer one column label to one hot label
def one_hot(y_):
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    y_ = y_.reshape(len(y_))
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [3]:
#  Data loading
feature = sc.loadmat("S1_nolabel6.mat")
all = feature['S1_nolabel6']
print (all.shape)

np.random.shuffle(all)   # mix eeg_all

final=2800*10
all=all[0:final]
feature_all =all[:,0:64]
label=all[:,64:65]

#z-score
feature_all=preprocessing.scale(feature_all)
ft = feature_all
no_fea=feature_all.shape[-1]
label_all=one_hot(label)
lt = label_all
print (label_all.shape)

(29738, 65)
(28000, 6)


In [4]:
#loading models
cnnmodel = load_model('cnnmodel.hdf5')
rnnmodel = load_model('rnnmodel.hdf5')
fcc = load_model('fcc.hdf5')
xgbmodel = XGBClassifier()
xgbmodel = pickle.load(open("xgbmodel.pickle.dat", "rb"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [5]:
def get_morse(sequence):
    #getting output from models
    get_cnn_output = K.function([cnnmodel.layers[0].input],[cnnmodel.layers[5].output])
    get_rnn_output = K.function([rnnmodel.layers[0].input],[rnnmodel.layers[0].output])
    get_fcc_output = K.function([fcc.layers[0].input],[fcc.layers[0].output])

    cnn_out = []
    train_fea_re = sequence.reshape(1, 64, 1).astype('float32')
    out = get_cnn_output([train_fea_re])
    cnn_out.append(out[0][0])
    rnn_out = []
    train_fea_re = sequence.reshape([1,1,64]).astype('float32')
    out = get_rnn_output([train_fea_re])
    rnn_out.append(out[0][0])
    fcc_out = []
    train = []
    train.extend(cnn_out[0])
    train.extend(rnn_out[0])
    train = np.asarray(train)
    train_fea_re = train.reshape(1,300).astype('float32')
    out = get_fcc_output([train_fea_re])
    fcc_out.append(out[0][0])  
    xgb_out = ''
    train = np.asarray(fcc_out[0])
    train_fea_re = train.reshape(1,30).astype('float32')
    out = xgbmodel.predict(train_fea_re)
    if out == 3:
        xgb_out = xgb_out + '/'
    else:
        xgb_out = xgb_out + (str)(out[0]-1)
    return(xgb_out)

In [9]:
#test input drivers
print(len(ft),len(lt))
ltt = []
for i in range(lt.shape[0]):
    ltt.append(np.argmax(lt[i]))
y1 = [ft[x] for x in range(len(ft)) if ltt[x] == 1]
y2 = [ft[x] for x in range(len(ft)) if ltt[x] == 2]
y3 = [ft[x] for x in range(len(ft)) if ltt[x] == 3]
y4 = [ft[x] for x in range(len(ft)) if ltt[x] == 4]
y5 = [ft[x] for x in range(len(ft)) if ltt[x] == 5]

label_b_m = {
    '0': y1,
    '1': y2,
    '/': y3,
}
sequence = random.choice(label_b_m['0'])
print(get_morse(sequence))
sequence = random.choice(label_b_m['1'])
print(get_morse(sequence))
sequence = random.choice(label_b_m['/'])
print(get_morse(sequence))

28000 28000
0
1
/
